In [15]:
from ttrtypes import BType,Pred,PType,Possibility,Ty,RecType,SingletonType
from records import Rec
from utils import show

In [2]:
M = Possibility('M')
M1 = Possibility('M1')
T = BType('T')
T.judge('c')
T.in_poss(M).judge('a')
T.in_poss(M1).judge('b')
print(show(M))
print(show(M1))


M:
_____________________________________________
T: [c, a]
_____________________________________________


M1:
_____________________________________________
T: [c, b]
_____________________________________________



In [3]:
T1 = BType('T1')
T1.learn_witness_condition(lambda x: len(x)==2)
print(T1.query('aa'))
print(T1.in_poss(M).query('aa'))

True
True


In [4]:
T.learn_witness_type(T1)
print(T.query('aa'))
print(T.in_poss(M).query('aa'))
print(T.in_poss(M1).query('bb'))

True
True
True


In [5]:
Ind = BType('Ind')
Ind.judge('sam')
Ind.judge('kim')
happy = Pred('happy',[Ind])
whistle = Pred('whistle',[Ind])
happySam = PType(happy,['sam'])
whistleSam = PType(whistle,['sam'])
whistleSam.in_poss(M).judge('e')
whistleSam.in_poss(M1).judge('e1')
happySam.learn_witness_type(whistleSam)
print(happySam.in_poss(M).query('e'))
print(happySam.in_poss(M1).query('e1'))
print(show(M))
print(show(M1))

True
True

M:
_____________________________________________
happy(sam): [e]
whistle(sam): [e]
T: [c, a, aa]
T1: [aa]
_____________________________________________


M1:
_____________________________________________
happy(sam): [e1]
whistle(sam): [e1]
T: [c, b, aa, bb]
T1: [aa, bb]
_____________________________________________



In [6]:
M2 = Possibility('M2')
smile = Pred('smile',[Ind])
smileKim = PType(smile,['kim'])
happyKim = PType(happy,['kim'])
smileKim.in_poss(M2).judge('e2')
happy.learn_witness_fun(lambda args: PType(smile,args))
happyKim.in_poss(M2).query('e2')

True

In [24]:
Triple = BType('Triple')
Triple.learn_witness_condition(lambda x: len(x)==3)
Bot = BType('Bot')
Bot.learn_witness_condition(lambda x: not x==x)
threesome = Pred('threesome',[Ind])
threesome.learn_witness_fun(lambda args: RecType({'x':SingletonType(Ind,args[0]),
                                                  'type':SingletonType(Ty,Triple)}) 
                            if Triple.query(args[0]) else Bot) 
                                                
Ind.judge('aaa')
Ind.judge('aa')
r = Rec({'x':'aaa',
        'type':Triple})
r_aa = Rec({'x':'aa',
            'type':Triple})
three_aaa = PType(threesome, ['aaa'])
print(three_aaa.query(r))
three_aa = PType(threesome,['aa'])
print(three_aa.query(r_aa))

True
False
